In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
def scraping_ecofin(champ):

  liens = []

  for debut in range(0, champ+10, 10):

    url = f"https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=33&submit_y=16&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=2025-02-24&filterDateTo=2025-03-31&filterFrench=French&userSearch=0&testlimitstart={debut}"

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    tables = soup.findAll('table', class_='ts')
    for table in tables:
      a_tags = table.findAll('a')
      for a_tag in a_tags:
        href = a_tag.get("href")
        if href:
            href = "https://www.agenceecofin.com"+ href
            if href not in liens:
              liens.append(href)
  return liens

In [ ]:
liens = scraping_ecofin(1050)

<ipython-input-2-5f2b7526be1f>:13: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  tables = soup.findAll('table', class_='ts')
<ipython-input-2-5f2b7526be1f>:15: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  a_tags = table.findAll('a')


In [ ]:
liens

In [ ]:
import re
def scraping_pages(url):

  texte = []

  page = requests.get(url)

  soup = BeautifulSoup(page.text, 'html')

  divs = soup.findAll("div", class_="itemIntroText")
  for div in divs:
    a_tags = div.findAll("p", class_="texte textearticle")
    for a_tag in a_tags:
      text_content = a_tag.text
      text_content = re.sub(r'[\[\]\,\(\)\"\\\']', '', text_content)
      texte.append(text_content)
  return texte

In [ ]:
contenu = []
for lien in liens:
  contenu.append(scraping_pages(lien))

In [ ]:
import pandas as pd

ta = {'Lien':liens, 'Texte':contenu}
df = pd.DataFrame(ta)

df.to_csv('my_data.csv', index=False)

In [ ]:
df.head()